In [26]:
%%capture
%pip install pandas

In [27]:
import pandas as pd

In [28]:
df = pd.read_csv('raw/combined-original.csv')

In [29]:
print(df.head())

   Unnamed: 0  Year  Month  DayofMonth  DayOfWeek  DepTime  CRSDepTime  \
0           0  1987     11          28          6   2042.0        2025   
1           1  1987     10          22          4   1805.0        1800   
2           2  1987     11          19          4    657.0         655   
3           3  1987     11          23          1   1848.0        1835   
4           4  1987     11          25          3    703.0         655   

   ArrTime  CRSArrTime UniqueCarrier  ...  TaxiIn TaxiOut  Cancelled  \
0   2105.0        2053            PI  ...     NaN     NaN          0   
1   1933.0        1905            HP  ...     NaN     NaN          0   
2    821.0         804            PI  ...     NaN     NaN          0   
3   1944.0        1920            DL  ...     NaN     NaN          0   
4    828.0         821            AA  ...     NaN     NaN          0   

   CancellationCode  Diverted  CarrierDelay  WeatherDelay NASDelay  \
0               NaN         0           NaN         

In [30]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [31]:
df.rename(columns={'DayOfWeek': 'DiaSemana', 'UniqueCarrier': 'CompanhiaResponsavel'}, inplace=True)

In [32]:
df.rename(columns={'DayofMonth': 'Day'}, inplace=True)
df['Data'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
df.drop(['Year', 'Month', 'Day'], axis=1, inplace=True)

In [33]:
# quero ver se Data possui algum NaN

print(df['Data'].isnull().sum())

# não possui

0


In [34]:
# Acabei percebendo que CancellationCode só possui valores NaN

df.drop(['CancellationCode'], axis=1, inplace=True)

In [35]:
# Também reparei que consequentemente nenhum voo foi cancelado

df.drop(['Cancelled'], axis=1, inplace=True)

In [36]:
# Existem colunas vindo como float porém no formato hhmm, bora transformar em hh:mm e manter os NaN como NaN

def format_time(val):
    if pd.isna(val):
        return val
    onlynumber = str(int(val)).zfill(4)
    return onlynumber[:2] + ":" + onlynumber[2:] + ":00"

In [37]:
df['HoraPartida'] = df['DepTime'].apply(format_time)
df['HoraPartidaPrevista'] = df['CRSDepTime'].apply(format_time)
df['HoraChegada'] = df['ArrTime'].apply(format_time)
df['HoraChegadaPrevista'] = df['CRSArrTime'].apply(format_time)
df.drop(['DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime'], axis=1, inplace=True)

In [38]:
# Vamos checar a relação entre esses valores:

df[['TaxiIn', 'TaxiOut', 'Diverted', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay',  'HoraPartidaPrevista', 'HoraPartida', 'HoraChegadaPrevista', 'HoraChegada']]

# Como podemos ver a partir das linhas 21997 e 21999:
# - Os valores de atraso dizem respeito ao atraso à hora de partida
# - Também existe um delay adicional não calculado
# - Os valores de TaxiIn e TaxiOut já estão inclusos na hora prevista, ou seja, não são atrasos

,TaxiIn,TaxiOut,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,HoraPartidaPrevista,HoraPartida,HoraChegadaPrevista,HoraChegada
0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,20:25:00,20:42:00,20:53:00,21:05:00
1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,18:00:00,18:05:00,19:05:00,19:33:00
2,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,06:55:00,06:57:00,08:04:00,08:21:00
3,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,18:35:00,18:48:00,19:20:00,19:44:00
4,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,06:55:00,07:03:00,08:21:00,08:28:00
...,...,...,...,...,...,...,...,...,...,...,...,...
21995,5.0,10.0,0,NaN,NaN,NaN,NaN,NaN,07:30:00,07:31:00,09:05:00,09:06:00
21996,2.0,8.0,0,NaN,NaN,NaN,NaN,NaN,15:30:00,15:41:00,16:30:00,16:36:00
21997,6.0,28.0,0,129.0,0.0,0.0,0.0,0.0,12:00:00,14:22:00,16:50:00,18:59:00
21998,2.0,11.0,0,NaN,NaN,NaN,NaN,NaN,16:40:00,16:44:00,17:45:00,17:46:00


In [39]:
print(df['TaxiIn'].describe())
print(df['TaxiIn'].value_counts())
print(df['TaxiIn'].isnull().sum())

count    13996.000000
mean         6.626107
std         25.262758
min          0.000000
25%          4.000000
50%          5.000000
75%          7.000000
max       1449.000000
Name: TaxiIn, dtype: float64
TaxiIn
5.0     2416
4.0     2385
3.0     2074
6.0     1636
7.0     1177
        ... 
90.0       1
67.0       1
96.0       1
61.0       1
99.0       1
Name: count, Length: 71, dtype: int64
8004


In [40]:
df.columns

Index(['DiaSemana', 'CompanhiaResponsavel', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Diverted', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay',
       'LateAircraftDelay', 'Data', 'HoraPartida', 'HoraPartidaPrevista',
       'HoraChegada', 'HoraChegadaPrevista'],
      dtype='object')

In [41]:
df.to_csv('trusted/combined.csv', index=False)